In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [17]:
df = pd.read_excel('bases_analises/df_analise2.xlsx')

df['cod_ibge'] = df['cod_ibge'].astype(str).str.strip()


In [8]:
# Load the shapefile
gdf_rgi = gpd.read_file('SP_RG_Imediatas_2024/SP_RG_Imediatas_2024.shp')

In [10]:
gdf_rgi.head()

,CD_RGI,NM_RGI,CD_RGINT,NM_RGINT,CD_UF,NM_UF,SIGLA_UF,CD_REGIA,NM_REGIA,SIGLA_RG,AREA_KM2,geometry
0,350048,Amparo,3510,Campinas,35,São Paulo,SP,3,Sudeste,SE,868.247,"POLYGON ((-46.62094 -22.72846, -46.62076 -22.7..."
1,350045,Araras,3510,Campinas,35,São Paulo,SP,3,Sudeste,SE,1380.624,"POLYGON ((-47.18375 -22.29273, -47.18316 -22.2..."
2,350046,Rio Claro,3510,Campinas,35,São Paulo,SP,3,Sudeste,SE,1391.298,"POLYGON ((-47.46755 -22.41402, -47.46752 -22.4..."
3,350043,Mogi Guaçu,3510,Campinas,35,São Paulo,SP,3,Sudeste,SE,1903.020,"POLYGON ((-46.99216 -22.54742, -46.99247 -22.5..."
4,350044,São João da Boa Vista,3510,Campinas,35,São Paulo,SP,3,Sudeste,SE,3621.346,"POLYGON ((-46.75113 -22.29902, -46.75133 -22.2..."


In [11]:
# carregar os shapefiles
gdf_mun = gpd.read_file('SP_Municipios_2024/SP_Municipios_2024.shp')  # precisa desse!

# garantir mesmo sistema de coordenadas
gdf_mun = gdf_mun.to_crs(gdf_rgi.crs)


In [12]:
gdf_mun_rgi = gpd.sjoin(
    gdf_mun,
    gdf_rgi.loc[:, ['CD_RGI', 'geometry']],  # usar loc para garantir
    how='left',
    predicate='intersects'
)

In [18]:
gdf_mun_rgi['CD_MUN'] = gdf_mun_rgi['CD_MUN'].astype(str).str.strip()

df_mun_rgi = gdf_mun_rgi[['CD_MUN', 'CD_RGI_left','NM_RGI']].drop_duplicates()

In [19]:
df_regioes = df.merge(
    df_mun_rgi,
    left_on='cod_ibge',
    right_on='CD_MUN',
    how='left'
)

In [20]:
df_regioes


,ano,cod_ibge,emprego_formal,emp_c_sal,massa_rendimentos_num,qtd_formal_Administração_Pública,qtd_formal_Agropecuária,qtd_formal_Alojamento_e_Alimentação,qtd_formal_Artes_e_Cultura,qtd_formal_Atividades_Imobiliárias,...,perc_Pes_PBF,perc_PCD_BPC,perc_Ido_BPC,perc_Pop_65_mais,perc_65_menos_Idoso_BPC,perc_cadunicoFam_PBF,perc_cadunicoF_CAD_EP,CD_MUN,CD_RGI_left,NM_RGI
0,2018,3500105,10445,9793,21389288.64,996,692,275,20,31,...,0.027839,0.011543,0.006029,0.155839,0.149810,0.057258,0.019410,3500105,350019,Adamantina - Lucélia
1,2018,3500204,854,828,1628663.10,309,318,2,9,0,...,0.147568,0.005850,0.005971,0.151146,0.145175,0.093331,0.080367,3500204,350025,São José do Rio Preto
2,2018,3500303,6984,6791,16726576.00,855,1343,133,27,9,...,0.133088,0.009530,0.005238,0.099051,0.093813,0.130297,0.080935,3500303,350044,São João da Boa Vista
3,2018,3500402,983,957,2018178.53,281,206,29,1,0,...,0.073180,0.006423,0.009854,0.158523,0.148668,0.082357,0.056873,3500402,350044,São João da Boa Vista
4,2018,3500501,4605,4476,9361612.67,861,55,1201,109,2,...,0.048411,0.008681,0.007198,0.125231,0.118034,0.122272,0.079073,3500501,350048,Amparo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,2020,3557006,19192,16953,51887106.26,2431,47,1169,79,47,...,0.083975,0.004977,0.004579,0.088955,0.084377,0.158841,0.148577,3557006,350003,Sorocaba
1931,2020,3557105,26761,24148,58632076.82,2427,204,707,154,198,...,0.072158,0.013172,0.011087,0.129283,0.118196,0.121266,0.121413,3557105,350027,Votuporanga
1932,2020,3557154,579,554,1423809.76,245,195,2,0,0,...,0.104297,0.005534,0.001237,0.134375,0.133138,0.066901,0.039975,3557154,350023,Birigui - Penápolis
1933,2020,3557204,1772,1675,5109110.53,447,143,7,5,0,...,0.165358,0.010949,0.007452,0.120347,0.112895,0.158878,0.152065,3557204,350015,Ourinhos


In [21]:
print(df_regioes[['cod_ibge', 'CD_RGI_left']].head())
print("Quantidade de municípios sem RGI:", df_regioes['CD_RGI_left'].isna().sum())


  cod_ibge CD_RGI_left
0  3500105      350019
1  3500204      350025
2  3500303      350044
3  3500402      350044
4  3500501      350048
Quantidade de municípios sem RGI: 0


In [22]:
df_regioes.rename(columns={'CD_RGI_left': 'cod_regiao'}, inplace=True)
df_regioes.rename(columns={'NM_RGI': 'nome_regiao'}, inplace=True)


In [ ]:
# Exporta o DataFrame para CSV
#df_regioes.to_csv('df_regioes.csv', index=False, encoding='utf-8')